<a href="https://colab.research.google.com/github/SaarthakG-Dtu/apnacollege-demo/blob/main/UAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -U ultralytics wandb
from ultralytics import YOLO
model=YOLO('best.pt')
results=model.predict('forward',save=TRUE)




FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'